In [ ]:
from pathlib import Path
import numpy as np
import numpy.typing as npt
import h5py
import matplotlib.pyplot as plt
from topostats.io import LoadScans
from topostats.plotting import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()
vmin = -3
vmax = 4
square_markersize_figsize_20 = 3.8

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/20250528_RA_dose_0GY_picoz")
data_dir = base_dir / "output_fig_nodestats"
figure_dir = Path("/Users/sylvi/topo_data/topostats_2/figures/fig-ordered-tracing")
assert figure_dir.exists()
assert data_dir.exists()

image_filename = "20250528_picoz_0RA_tip_0.0_00019"

image_topostats_file_path = data_dir / "processed" / (f"{image_filename}.topostats")
assert image_topostats_file_path.exists()

loadscans = LoadScans(img_paths=[image_topostats_file_path], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict

image_dict = loadscans_dict[image_filename]
print(image_dict.keys())
image = image_dict["image"]
p2nm = image_dict["pixel_to_nm_scaling"]
print(f"pixel to nm scaling: {p2nm} nm/pixel")
print(f"image size: {image.shape} px, {image.shape[0]*p2nm:.2f} x {image.shape[1]*p2nm:.2f} nm")

image_disordered_traces = image_dict["disordered_traces"]["above"]
image_nodestats = image_dict["nodestats"]["above"]
image_ordered_tracing = image_dict["ordered_traces"]["above"]

image_nodestats_images_grain1 = image_nodestats["images"]["grain_1"]
image_nodestats_images_grain1_grain = image_nodestats_images_grain1["grain"]
image_nodestats_images_grain1_grain_image = image_nodestats_images_grain1_grain["grain_image"]

image_disordered_traces_grain1 = image_disordered_traces["grain_1"]
image_disordered_traces_grain1_branch_indexes = image_disordered_traces_grain1["branch_indexes"]

In [ ]:
# colouring by branch index
# create a mask the the branch indexes
image_disordered_traces_grain1_branch_indexes_mask = np.ma.masked_where(
    image_disordered_traces_grain1_branch_indexes == 0,
    image_disordered_traces_grain1_branch_indexes,
)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(image_disordered_traces_grain1_branch_indexes_mask, cmap="Accent", alpha=1)
plt.axis("off")
plt.savefig(figure_dir / f"i19-fig-ordered-tracing-branch-indexes-grain-1.png", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ordered_skeleton_mask = image_disordered_traces_grain1_branch_indexes.astype(bool)
ordered_skeleton_mask_ma = np.ma.masked_where(~ordered_skeleton_mask, ordered_skeleton_mask)
ax.imshow(ordered_skeleton_mask_ma, cmap="summer", alpha=1)

for node_id, node_image_data in image_nodestats_images_grain1["nodes"].items():
    print(f"node_id: {node_id}")
    node_branch_mask = node_image_data["node_branch_mask"]
    node_area_skeleton = node_image_data["node_area_skeleton"]
    node_avg_mask = node_image_data["node_avg_mask"]

    # plt.imshow(node_branch_mask)
    # plt.title(f"node_id: {node_id} branch mask")
    # plt.show()
    # plt.imshow(node_area_skeleton)
    # plt.title(f"node_id: {node_id} area skeleton")
    # plt.show()
    # plt.imshow(node_avg_mask)
    # plt.title(f"node_id: {node_id} avg mask")
    # plt.show()

    # make masked array for node branch mask
    node_branch_mask_ma = np.ma.masked_where(node_branch_mask == 0, node_branch_mask)
    ax.imshow(node_branch_mask_ma, cmap="cool", alpha=1)

In [ ]:
# colouring by molecule
cat_data_file = Path(
    "/Users/sylvi/topo_data/topostats_2/datasets/cats-images/output_fig_ordered_tracing_multi_molecule/processed/420231218_2ngSCcats.0_00179.topostats"
)

loadscans = LoadScans(img_paths=[cat_data_file], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict

cat_topleft = (0, 103)
cat_crop_size = (256, 256)

cat_image_dict = loadscans_dict[list(loadscans_dict.keys())[0]]

cat_p2nm = cat_image_dict["pixel_to_nm_scaling"]
cat_image = cat_image_dict["image"]
print(f"image size nm : {cat_image.shape[0]*cat_p2nm:.2f} x {cat_image.shape[1]*cat_p2nm:.2f} nm")
cat_crop = cat_image[
    cat_topleft[0] : cat_topleft[0] + cat_crop_size[0], cat_topleft[1] : cat_topleft[1] + cat_crop_size[1]
]
plt.imshow(cat_image, cmap=cmap, vmin=vmin, vmax=vmax)
plt.show()
plt.imshow(cat_crop, cmap=cmap, vmin=vmin, vmax=vmax)
plt.show()
cat_ordered_tracing = cat_image_dict["ordered_traces"]["above"]

cat_ordered_coords_mol0 = cat_ordered_tracing["grain_0"]["mol_0"]["ordered_coords"]
cat_ordered_coords_mol1 = cat_ordered_tracing["grain_0"]["mol_1"]["ordered_coords"]

# plot the crossings for the cat
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cat_crop, cmap=cmap, vmin=vmin, vmax=vmax)
cat_image_nodestats = cat_image_dict["nodestats"]["above"]
cat_image_nodestats_images_grain0 = cat_image_nodestats["images"]["grain_0"]
cat_image_nodestats_images_grain0_nodes = cat_image_nodestats_images_grain0["nodes"]
for node_id, node_image_data in cat_image_nodestats_images_grain0_nodes.items():
    print(f"node_id: {node_id}")
    node_branch_mask = node_image_data["node_branch_mask"]
    node_branch_mask_ma = np.ma.masked_where(node_branch_mask == 0, node_branch_mask)
    ax.imshow(node_branch_mask_ma, cmap="cool", alpha=1)
    print(node_branch_mask.shape)
plt.savefig(figure_dir / f"fig-ordered-tracing-crossings-cats-image", bbox_inches="tight")

# Plot separate molecules for the cat
cat_molecule_trace_linewidth = 3
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cat_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.plot(
    cat_ordered_coords_mol0[:, 1],
    cat_ordered_coords_mol0[:, 0],
    color="#2578B1",
    linewidth=cat_molecule_trace_linewidth,
)
ax.plot(
    cat_ordered_coords_mol1[:, 1],
    cat_ordered_coords_mol1[:, 0],
    color="#27B125",
    linewidth=cat_molecule_trace_linewidth,
)
plt.savefig(figure_dir / f"fig-ordered-tracing-molecules-cats-image", bbox_inches="tight")

In [ ]:
# overlapping_molecules_image_file = Path("/Users/sylvi/topo_data/topostats_2/datasets/overlapping-molecules/output/processed/20250721_MC1Rx_8ng_IncMg_ImageNI_2.0_00003.topostats")
overlapping_molecules_image_file = Path(
    "/Users/sylvi/topo_data/topostats_2/datasets/overlapping-molecules/output_short_crossings/processed/20250721_MC1Rx_8ng_IncMg_ImageNI_2.0_00003.topostats"
)

loadscans = LoadScans(img_paths=[overlapping_molecules_image_file], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict
overlapping_molecules_image_dict = loadscans_dict[list(loadscans_dict.keys())[0]]
overlapping_molecules_p2nm = overlapping_molecules_image_dict["pixel_to_nm_scaling"]
overlapping_molecules_image = overlapping_molecules_image_dict["image"]
plt.imshow(overlapping_molecules_image, cmap=cmap, vmin=vmin, vmax=vmax)
plt.show()


overlapping_grain_id = "grain_2"
overlapping_grain_bbox = overlapping_molecules_image_dict["splining"]["above"][overlapping_grain_id]["mol_0"]["bbox"]
overlapping_crop = overlapping_molecules_image[
    overlapping_grain_bbox[0] : overlapping_grain_bbox[2], overlapping_grain_bbox[1] : overlapping_grain_bbox[3]
]
overlapping_nodestats = overlapping_molecules_image_dict["nodestats"]["above"]
overlapping_nodestats_images_grain = overlapping_nodestats["images"][overlapping_grain_id]
overlapping_nodestats_images_grain_grain_image = overlapping_nodestats_images_grain["grain"]["grain_image"]
overlapping_disordered_traces = overlapping_molecules_image_dict["disordered_traces"]["above"]
overlapping_disordered_traces_grain = overlapping_disordered_traces[overlapping_grain_id]
overlapping_ordered_coords_grain = overlapping_molecules_image_dict["ordered_traces"]["above"][overlapping_grain_id]
overlapping_ordered_coords_grain_mol0 = overlapping_ordered_coords_grain["mol_0"]
overlapping_ordered_coords_grain_mol1 = overlapping_ordered_coords_grain["mol_1"]
overlapping_ordered_coords_grain_mol0_ordered_coords = overlapping_ordered_coords_grain_mol0["ordered_coords"]
overlapping_ordered_coords_grain_mol1_ordered_coords = overlapping_ordered_coords_grain_mol1["ordered_coords"]

# plot the fwhm
# grab the heights
overlapping_nodestats_stats_grain = overlapping_nodestats["stats"][overlapping_grain_id]
overlapping_nodestats_stats_grain_node1 = overlapping_nodestats_stats_grain["node_1"]
overlapping_nodestats_stats_grain_node1_branch_stats_0 = overlapping_nodestats_stats_grain_node1["branch_stats"]["0"]
overlapping_nodestats_stats_grain_node1_branch_stats_1 = overlapping_nodestats_stats_grain_node1["branch_stats"]["1"]
overlapping_nodestats_stats_grain_node1_branch_stats_0_heights = overlapping_nodestats_stats_grain_node1_branch_stats_0["heights"]
overlapping_nodestats_stats_grain_node1_branch_stats_1_heights = overlapping_nodestats_stats_grain_node1_branch_stats_1["heights"]
overlapping_nodestats_stats_grain_node1_branch_stats_0_fwhm = overlapping_nodestats_stats_grain_node1_branch_stats_0["fwhm"]
overlapping_nodestats_stats_grain_node1_branch_stats_1_fwhm = overlapping_nodestats_stats_grain_node1_branch_stats_1["fwhm"]
print(f"node 1 branch 0 fwhm: {overlapping_nodestats_stats_grain_node1_branch_stats_0_fwhm}")
print(f"node 1 branch 1 fwhm: {overlapping_nodestats_stats_grain_node1_branch_stats_1_fwhm}")
plt.plot(overlapping_nodestats_stats_grain_node1_branch_stats_0_heights, label="node 1 branch 0")
plt.plot(overlapping_nodestats_stats_grain_node1_branch_stats_1_heights, label="node 1 branch 1")
plt.axhline(overlapping_nodestats_stats_grain_node1_branch_stats_0_fwhm["half_maxs"][2], color="gray", linestyle="--", label="branch 0 half max")
plt.axhline(overlapping_nodestats_stats_grain_node1_branch_stats_1_fwhm["half_maxs"][2], color="black", linestyle="--", label="branch 1 half max")
plt.legend()
plt.show()

overlapping_nodestats_stats_grain_node2 = overlapping_nodestats_stats_grain["node_2"]
overlapping_nodestats_stats_grain_node2_branch_stats_0 = overlapping_nodestats_stats_grain_node2["branch_stats"]["0"]
overlapping_nodestats_stats_grain_node2_branch_stats_1 = overlapping_nodestats_stats_grain_node2["branch_stats"]["1"]
overlapping_nodestats_stats_grain_node2_branch_stats_0_heights = overlapping_nodestats_stats_grain_node2_branch_stats_0["heights"]
overlapping_nodestats_stats_grain_node2_branch_stats_1_heights = overlapping_nodestats_stats_grain_node2_branch_stats_1["heights"]
overlapping_nodestats_stats_grain_node2_branch_stats_0_fwhm = overlapping_nodestats_stats_grain_node2_branch_stats_0["fwhm"]
overlapping_nodestats_stats_grain_node2_branch_stats_1_fwhm = overlapping_nodestats_stats_grain_node2_branch_stats_1["fwhm"]
print(f"node 2 branch 0 fwhm: {overlapping_nodestats_stats_grain_node2_branch_stats_0_fwhm}")
print(f"node 2 branch 1 fwhm: {overlapping_nodestats_stats_grain_node2_branch_stats_1_fwhm}")
plt.plot(overlapping_nodestats_stats_grain_node2_branch_stats_0_heights, label="node 2 branch 0")
plt.plot(overlapping_nodestats_stats_grain_node2_branch_stats_1_heights, label="node 2 branch 1")
plt.axhline(overlapping_nodestats_stats_grain_node2_branch_stats_0_fwhm["half_maxs"][2], color="gray", linestyle="--", label="branch 0 half max")
plt.axhline(overlapping_nodestats_stats_grain_node2_branch_stats_1_fwhm["half_maxs"][2], color="black", linestyle="--", label="branch 1 half max")
plt.legend()
plt.show()


# plot branch crossings
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(overlapping_crop, cmap=cmap, vmin=vmin, vmax=vmax)
for node_id, node_image_data in overlapping_nodestats_images_grain["nodes"].items():
    print(f"node_id: {node_id}")
    node_branch_mask = node_image_data["node_branch_mask"]
    node_branch_mask_ma = np.ma.masked_where(node_branch_mask == 0, node_branch_mask)
    ax.imshow(node_branch_mask_ma, cmap="cool", alpha=1)
plt.axis("off")
plt.show()

# Plot molecules
plt.imshow(overlapping_crop, cmap=cmap, vmin=vmin, vmax=vmax)
plt.plot(
    overlapping_ordered_coords_grain_mol0_ordered_coords[:, 1],
    overlapping_ordered_coords_grain_mol0_ordered_coords[:, 0],
    color="#2578B1",
    linewidth=cat_molecule_trace_linewidth,
)
plt.plot(
    overlapping_ordered_coords_grain_mol1_ordered_coords[:, 1],
    overlapping_ordered_coords_grain_mol1_ordered_coords[:, 0],
    color="#27B125",
    linewidth=cat_molecule_trace_linewidth,
)
plt.show()